In [0]:
import dlt
from pyspark.sql.functions import col

# Set the catalog and schema to the target (Silver) database
spark.sql("USE CATALOG principal_lab_db")
spark.sql("USE SCHEMA dev_silver")

# Load table metadata for dim_policies from config_dev.table_lookup
config = spark.table("principal_lab_db.config_dev.table_lookup") \
    .filter(col("table_name") == "dim_policies") \
    .select("keys", "scd_type", "description") \
    .first()
# Extract configuration values
keys = config["keys"]              # list of key column names
scd_type = int(config["scd_type"]) # SCD type (expected 2 for Type 2)
dim_description = config["description"]

@dlt.view(name="policies_clean")
def policies_clean():
    # Read the Bronze snapshot table as a streaming DataFrame
    bronze_df = spark.readStream.table("principal_lab_db.dev_bronze.policies_bronze")
    # Apply any required cleaning or transformations (not shown for brevity)
    return bronze_df

# Create the target Silver dimension table
dlt.create_streaming_table(
    name="dim_policies",
    comment=dim_description,
    table_properties={"quality": "silver"}
)

# Apply SCD Type 2 changes from the source view to the target table
dlt.apply_changes(
    target="dim_policies",
    source="policies_clean",
    keys=keys,
    sequence_by=col("snapshot_date"),
    stored_as_scd_type=scd_type,  # use 2 for SCD Type 2
    track_history_except_column_list=["ingestion_ts", "source_file", "snapshot_date"]
)
